In [1]:
import numpy as np
from numba import cuda

In [2]:
###################
# iDivUp FUNCTION #
###################
def iDivUp(a, b):
    # Round a / b to nearest higher integer value
    a = np.int32(a)
    b = np.int32(b)
    return (a // b + 1) if (a % b != 0) else (a // b)

In [3]:
# --- CUDA kernel for vector addition
@cuda.jit
def deviceAdd(d_c, d_a, d_b, N):
    tid = cuda.threadIdx.x + cuda.blockIdx.x * cuda.blockDim.x
    if tid < N:
        d_c[tid] = d_a[tid] + d_b[tid]

In [4]:
########
# MAIN #
########

# --- Initialize parameters
N = 100000
BLOCKSIZE = 256
gridDim = iDivUp(N, BLOCKSIZE)

# --- Create random vectors on the CPU
h_a = np.random.randn(N).astype(np.float32)
h_b = np.random.randn(N).astype(np.float32)
h_c = np.empty_like(h_a)

# --- Allocate GPU device memory
d_a = cuda.to_device(h_a)
d_b = cuda.to_device(h_b)
d_c = cuda.device_array_like(h_a)

# --- Warmup execution
deviceAdd[gridDim, BLOCKSIZE](d_c, d_a, d_b, N)

# --- Measure execution time
start = cuda.event()
end = cuda.event()

start.record()
deviceAdd[gridDim, BLOCKSIZE](d_c, d_a, d_b, N)
end.record()
end.synchronize()

secs = cuda.event_elapsed_time(start, end) * 1e-3
print(f"Processing time = {secs:.6f}s")

# --- Copy results from device to host
d_c.copy_to_host(h_c)

# --- Verify the result
if np.array_equal(h_c, h_a + h_b):
    print("Test passed!")
else:
    print("Error!")


Processing time = 0.000210s
Test passed!
